In [7]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import yaml
import phacsl.utils.formats.csv_tools as csv_tools
import os.path
from collections import defaultdict

from tools_util import readModelInputs, getFacDict

In [8]:
#
# Point this at the run description
#
#runDesc = '/home/welling/git/pyRHEA_github/src/sim/week_run_OC.yaml'
runDesc = '/home/welling/git/pyRHEA_github/src/sim/week_run_ChicagoLand.yaml'

inputDict = readModelInputs(runDesc)
facDict = getFacDict(inputDict)

totD = defaultdict(int)
nFacD = defaultdict(int)
for fac, rec in facDict.items():
    nFacD[rec['category']] += 1
    totD[rec['category']] += rec['meanPop']['value']
print 'totD: ', totD
print 'nFacD: ', nFacD


totD:  defaultdict(<type 'int'>, {'SNF': 44029.96048152038, 'HOSPITAL': 13839.558332146706, 'VSNF': 2359.043835616, 'COMMUNITY': 9082963, 'LTACH': 528.550274452})
nFacD:  defaultdict(<type 'int'>, {'SNF': 351, 'HOSPITAL': 90, 'VSNF': 12, 'COMMUNITY': 2260, 'LTACH': 9})


In [ ]:
#
#  Point this at the .mpk file produced by parse_taumod_taumod_out.py
#
#fullDF = pd.read_msgpack('/home/welling/git/pyRHEA_github/src/tools/taumod_work6.mpk')
#fullDF = pd.read_msgpack('/home/welling/sshfshook/taumod_oc_mrsa_201811/work5/taumod_work5.mpk')
#fullDF = pd.read_msgpack('/home/welling/sshfshook/taumod_oc_mrsa_201811b/work1/taumod_b_work1.mpk')
#fullDF = pd.read_msgpack('/home/welling/sshfshook/taumod_oc_mrsa_201811b/work2/taumod_b_work2.mpk')
#fullDF = pd.read_msgpack('/home/welling/sshfshook/taumod_oc_mrsa_201811b/work3/taumod_b_work3.mpk')
#fullDF = pd.read_msgpack('/home/welling/sshfshook/taumod_oc_mrsa_201811b/work4/taumod_b_work4.mpk')
#fullDF = pd.read_msgpack('/home/welling/sshfshook/taumod_oc_mrsa_201811b/work9/taumod_work9.mpk')
fullDF = pd.read_msgpack('/home/welling/sshfshook/taumod_201810/work13/taumod_work13.mpk')
print fullDF.columns

In [ ]:
tierPairs = [('NURSINGHOME', 'NURSING'), ('HOSPITAL', 'HOSP'), ('HOSPITAL', 'ICU'),
            ('LTAC', 'LTAC'), ('LTACH', 'LTAC')]


In [ ]:
#loc, tier = 'PRES_333_H', 'HOSP'
#loc, tier = 'PRES_333_H', 'ICU'
#loc, tier = 'ADVO_4440_H', 'HOSP'
#loc, tier = 'ADVO_4440_H', 'ICU'
#loc, tier = 'ADVO_3435_L', 'LTAC'
#loc, tier = 'PRES_400_S', 'NURSING'
#loc, tier = 'PARK_2320_V', 'NURSING'
#loc, tier = 'CHAT_7050_S', 'NURSING'
loc, tier = '*', 'NURSING'


In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
plt.rcParams["figure.figsize"] = [16, 4.5]

def plotLoc(loc, tier, fullDF):
    locDF = fullDF[np.logical_and(fullDF['fac']==loc, fullDF['tier']==tier)]
    fig, axes = plt.subplots(1,2)
    axes[0].plot(locDF['samp'], locDF['newTau'], label='newTau')
    axes[0].plot(locDF['samp'], locDF['current_best_tau'], label='current_best')
    axes[0].plot(locDF['samp'], locDF['tau'], label='tau')
    axes[0].legend()
    axes[0].set_title('%s %s tau' % (loc, tier))
    axes[0].set_yscale('log')
    axes[1].plot(locDF['samp'], locDF['expected'], label='expected')
    axes[1].plot(locDF['samp'], locDF['prev_q1'], label='q1')
    axes[1].plot(locDF['samp'], locDF['prev_q2'], label='median')
    axes[1].plot(locDF['samp'], locDF['prev_q3'], label='q3')
    axes[1].plot(locDF['samp'], locDF['prevalence'], label='prevalence')
    axes[1].legend()
    if loc == '*':
        axes[1].set_title('prevalence')
    else:
        axes[1].set_title('prevalence (meanPop=%s)' % facDict[loc]['meanPop']['value'])
    #axes[1].set_yscale('log')
    plt.show()

In [ ]:
maxSamp = fullDF['samp'].max()
allTiers = set([tier for loc, tier in tierPairs])
for tier in allTiers:
    plotLoc('*', tier, fullDF[fullDF['samp']>maxSamp-15])

In [ ]:
# Tau plots
#ctg, tier = 'NURSINGHOME', 'NURSING'
#ctg, tier = 'HOSPITAL', 'HOSP'
#ctg, tier = 'HOSPITAL', 'ICU'
#ctg, tier = 'LTAC', 'LTAC'
ctg, tier = 'LTACH', 'LTAC'
#ctg, tier = 'VSNF', 'SKILNRS'
#ctg, tier = 'VSNF', 'VENT'
#ctg, tier = 'VSNF', 'NURSING'
count = 0
candidateL = []
for loc, rec in facDict.items():
    if rec['category'] == ctg:
        candidateL.append((loc, tier))
random.seed(1234) # for consistency
random.shuffle(candidateL)
for loc, tier in candidateL[:12]:
    plotLoc(loc, tier, fullDF)


In [ ]:
zeroD = defaultdict(list)
oneD = defaultdict(list)
goodD = defaultdict(list)
invalidD = defaultdict(list)
maxSamp = fullDF['samp'].max()
for cat, tier in tierPairs:
    for fac, rec in facDict.items():
        if rec['category'] == cat:
            locDF = fullDF[np.logical_and(fullDF['fac']==fac, fullDF['tier']==tier)]
            #print fac, tier, len(locDF)
            sampDF = locDF[locDF['samp']>=maxSamp-15]
            tau = sampDF.mean()['tau']
            #print fac, tier, sampDF.mean()['tau']
            if np.isnan(tau):
                invalidD[(cat, tier)].append(fac)
            elif tau > 0.8:
                oneD[(cat, tier)].append(fac)
            elif tau < 1.0e-6:
                zeroD[(cat, tier)].append(fac)
            else:
                goodD[(cat, tier)].append(fac)
print 'cat\ttier\tgood\tzero\tone\tnan'
for cat, tier in tierPairs:
    print '%s\t%s\t%s\t%s\t%s\t%s' % (cat[:6], tier, len(goodD[(cat,tier)]),
                         len(zeroD[(cat, tier)]), len(oneD[(cat, tier)]), len(invalidD[(cat,tier)]))
            


## Estimating tau for an ICU ##

```
Prevalence: 0.03
mean ICU LOS: 3.67
Ward size: 12 beds max
mean exposures: 12 * 3.67 * 0.03 = 1.32
1.32 * tau = 0.03
tau = 0.023
```


## Estimating tau for a HOSP ward ##

```
Prevalence: 0.05
mean ICU LOS: 5.41
Ward size: 20 beds max
mean exposures: 20 * 5.41 * 0.05 = 5.41
5.41 * tau = 0.05
tau = 0.0092
```


## Estimating tau for a NURSING ward ##

```
Prevalence: 0.015
mean ICU LOS: 118.8
facility size: 125.4 beds mean
mean exposures: 125.4 * 118.8 * 0.015 = 223.46
223.46 * tau = 0.015
tau = 6.7e-5
```


In [ ]:
vL = []
for fac, rec in facDict.items():
    if rec['category'] != 'NURSINGHOME':
        continue
    locDF = fullDF[np.logical_and(fullDF['fac']==fac, fullDF['tier']=='NURSING')]
    sampDF = locDF[locDF['samp']>=maxSamp-15]
    tau = sampDF.mean()['tau']
    vL.append((rec['nBeds']['value'], tau, fac))
vL.sort()
#print vL
fig, axes = plt.subplots(1,1)
axes.plot([v[0] for v in vL],[v[1] for v in vL], 'r+')
axes.set_yscale('log')
axes.set_xlabel('nBeds')
axes.set_ylabel('tau')
axes.set_title('SNF NURSING tau vs nBeds')
#axes.set_ylim(1.0e-9, 1.0e-1)
plt.show()
            